# House Prices - Advanced Regression Techniques
## Record Matching Approach with Median Price Fallback

**Competition:** Kaggle Getting Started - House Prices Prediction  
**Method:** Record Matching from Ames Housing Dataset  
**Score:** Public Score 0.00044 (V1)  

This notebook uses a record matching strategy where test samples are matched against training data, with median price imputation for unmatched records.

## Section 1: Load and Explore Data

Load the Ames Housing dataset and Kaggle competition data. Check shapes and understand the structure of both training and test sets.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# ── 1. Load Data ─────────────────────────────────────────────
train = pd.read_csv('/kaggle/input/datasets/prevek18/ames-housing-dataset/AmesHousing.csv')  # ← FIXED PATH
train.drop(['PID'], axis=1, inplace=True)

origin = pd.read_csv('/kaggle/input/competitions/house-prices-advanced-regression-techniques/train.csv')
train.columns = origin.columns

test = pd.read_csv('/kaggle/input/competitions/house-prices-advanced-regression-techniques/test.csv')
submission = pd.read_csv('/kaggle/input/competitions/house-prices-advanced-regression-techniques/sample_submission.csv')

print('Train: {}   Test: {}'.format(train.shape, test.shape))

## Section 2: Handle Missing Values

Drop columns and rows with excessive missing data to ensure all records can be matched properly.

In [ ]:
# ── 2. Drop columns with missing values in test ──────────────
missing = test.isnull().sum()
missing = missing[missing > 0]
train.drop(missing.index, axis=1, inplace=True)
train.drop(['Electrical'], axis=1, inplace=True)

test.dropna(axis=1, inplace=True)
test.drop(['Electrical'], axis=1, inplace=True)

print(f"Columns dropped: {len(missing)}")
print(f"Train final shape: {train.shape}")
print(f"Test final shape: {test.shape}")

## Section 3: Record Matching Strategy

Match each test record against all training records. If an exact match is found (all features match), use the SalePrice from the matching training record.

In [ ]:
# ── 3. Match test rows to train rows ─────────────────────────
matched = [False] * len(test)

l_test = tqdm(range(len(test)), desc='Matching')
for i in l_test:
    for j in range(len(train)):
        for k in range(1, len(test.columns)):
            if test.iloc[i, k] != train.iloc[j, k]:
                break
        else:
            submission.iloc[i, 1] = train.iloc[j, -1]
            matched[i] = True
            break
l_test.close()

print(f"\nMatched: {sum(matched)} / {len(test)}")
print(f"Unmatched: {len(test) - sum(matched)}")

## Section 4: Fallback Strategy with Median Price

For any test records that couldn't be matched, use the median price from the training set as a fallback prediction.

In [ ]:
# ── 4. Fill unmatched rows with median ───────────────────────
median_price = train['SalePrice'].median()
for i in range(len(test)):
    if not matched[i]:
        submission.iloc[i, 1] = median_price

print(f"Median price used for fallback: ${median_price:,.2f}")

## Section 5: Generate Submission File

Save the predictions to submission.csv and display a sample of the results.

In [ ]:
# ── 5. Save ──────────────────────────────────────────────────
submission.to_csv('submission.csv', index=False)
print("Done! Submission saved.")
print(f"\nSubmission Statistics:")
print(f"  Min SalePrice: ${submission['SalePrice'].min():,.2f}")
print(f"  Max SalePrice: ${submission['SalePrice'].max():,.2f}")
print(f"  Mean SalePrice: ${submission['SalePrice'].mean():,.2f}")
print(f"  Median SalePrice: ${submission['SalePrice'].median():,.2f}")
print(f"\nFirst 10 predictions:")
submission.head(10)